In [47]:
from prov_parameters import generator
import numpy as np
import pandas as pd
import ixmp as ix
import message_ix

In [2]:
mp = ix.Platform()

msg = message_ix.Scenario(mp, 'MESSAGEix', 'baseline',
                          version = 'new')

2021-11-23 10:22:22,667  INFO at.ac.iiasa.ixmp.Platform:146 - Welcome to the IX modeling platform!
2021-11-23 10:22:22,669  INFO at.ac.iiasa.ixmp.Platform:147 -  connected to database 'jdbc:hsqldb:file:/home/noah/.local/share/ixmp/localdb/default' (user: ixmp)...


In [6]:
msg.read_excel('MESSAGE_CA__test__v5.xlsx',
               add_units = False,
               commit_steps = False,
               init_items = True)

In [7]:
msg.check_out()

In [64]:
provinces = ['Alberta',
            'BritishColumbia',
            'Manitoba',
            'NewBrunswick',
            'NewfoundlandandLabrador',
            'NorthwestTerritories',
            'NovaScotia',
            'Nunavut',
            'Ontario',
            'PrinceEdwardIsland',
            'Quebec',
            'Saskatchewan',
            'Yukon']

msg.add_spatial_sets({'country': {'Canada': {'province': provinces}}})

In [17]:
# All (non-empty) parameters of the Canadian model
total_pars = [par for par in msg.par_list() if not msg.par(par).empty]

# Parameters already found with data
found_pars = ['demand', 'historical_activity', 'historical_new_capacity']

# Economic parameters to skip for now
econ_pars = ['MERtoPPP', 
             'abs_cost_activity_soft_lo',
             'abs_cost_activity_soft_up',
             'cost_MESSAGE',
             'depr',
             'drate',
             'esub',
             'fix_cost',
             'gdp_calibrate',
             'grow',
             'interestrate',
             'inv_cost',
             'kgdp',
             'kpvs',
             'level_cost_activity_soft_lo',
             'level_cost_activity_soft_up',
             'level_cost_new_capacity_soft_up',
             'price_MESSAGE',
             'relation_cost',
             'resource_cost',
             'var_cost']

# Filter out un-needed parameters
filtered = [x for x in total_pars if x not in found_pars if x not in econ_pars]
filtered.sort()

In [66]:
## Generate provincial dataframes
# Create dictionary to store node and year data for parameters
pars_to_change = {}

# Iterate through parameters
for par in filtered:
    
    node_stuff = []
    year_stuff = []

    # Iterate through columns of parameter dataframe
    for col in msg.par(par).columns:

        # Check for node column (or some variation including string 'node')
        if 'node' in col:
            node_stuff.append(col)
        
        # Check for year column, make sure it contains integers
        if 'year' in col and msg.par(par).dtypes[col] == 'int64':
            year_stuff.append(col)

    if len(node_stuff) != 0:
        # Add parameter and column values to dictionary
        pars_to_change[par] = [node_stuff, year_stuff]

In [69]:
# Create dictionary with key = "parameter name" and value = <provincial dataframe>
dfs = dict.fromkeys(pars_to_change.keys())

for par in dfs:    
    # Copy canadian dataframe
    can_df = msg.par(par).copy()
    
    # Only need data for years after (and including) 2010
    for year_data in pars_to_change[par][1]:
        can_df = can_df[can_df[year_data] >= 2010]
    
    dfs[par] = can_df
    
    for prov in provinces:
        
        # Create copy of Canadian dataframe
        df = msg.par(par).copy()
        
        # Only need data for years after (and including) 2010
        for year_data in pars_to_change[par][1]:
            
            df = df[df[year_data] >= 2010]

        # Change node values to province
        for node_data in pars_to_change[par][0]:
            df[node_data] = region

        # Append provincial data to parameter dataframe
        dfs[par] = dfs[par].append(df)